# The `JADE` code: Tutorial #1

*Author:* Mara Attia

***

This tutorial explains the basics of the `JADE` code. It represents a practical guide, showing you how to get acquainted with the code. If you want to know more about the physics behind the model, you can refer to [Attia et al. (2021)](https://ui.adsabs.harvard.edu/abs/2021A%26A...647A..40A/abstract). If you make use of the `JADE` code for a scientific publication, please cite the latter article.

We will focus here on the most fundamental level at which you can use the `JADE` code, namely **forward modeling** with a **single simulation at a time**. To employ the terminology of the `JADE` use cases (see [Attia et al. 2025]()), we will go through **FD1**, **FA1**, **FA3**, and **FC**. Their meaning will become clear as you progress in this tutorial.

## Basic usage

Using the `JADE` code is pretty straightforward. It follows the simple rule:

> **One simulation = One input file**.

Consequently, if you get familiar with input files, you are already halfway there. Several example input files are provided within the package, for the sake of the tutorials.

An important element to note is that **all input files should always be located in the `input/` folder**. You can classify your input files in different folders (and even nested folders), but everything should be inside `input/`. 

The structure and contents of the input files are documented in the examples. **It is important to read the input documentation, all of it, before launching any simulation**. Until you get familiar with creating your own, you can duplicate the example input files that are the closest to your intended usage, and modify the parameters.

Once you have your input file, let us say `input/my_folder/my_simulation.txt`, you can launch the simulation by running `routines/main.py` with that file as an argument. Concretely, in a terminal, once you are in the `JADE` base folder, run the following:

    cd routines
    python main.py my_folder/my_simulation.txt
    
Do not add `input/` before the path to the input file, **said path should always be relative to `input/`**. As the simulation progresses, information will be printed by `JADE` on the terminal. If you want to save these information (the “log” of the simulation) in a file instead, you can redirect the terminal standard output:

    python main.py my_folder/my_simulation.txt > path/to/my_simulation.log
    
*Note:* the simulation log is not the output that you will use for doing science. It is for information and housekeeping purposes only.

***

**The proper simulation output will always be located in the `saved_data/` folder, in the form of `.npz` files**. You can define a specific directory for your output (relative to `saved_data/`) in the input file, by setting the `output_dir` field. 

Good practice dictates consistency between your input/output directory structure. If we take our previous example input file `input/my_folder/my_simulation.txt`, the output directory should be set to `output_dir = my_folder/my_simulation`. This way, your output files will be located in `saved_data/my_folder/my_simulation/` and will be called `simulation_name_001.npz`, `simulation_name_002.npz`, etc, where 
- `simulation_name` is defined in the input file in the `name` field,
- the number of `.npz` files is defined by `output_freq`, the saving frequency. For example, a value of `output_freq = 20` will save an `.npz` file every 20%, so you will have five of them. *Note:* this might differ if you resume a halted simulation using the `reuse` field.

The `.npz` files are the ones you would analyze for doing science. You can try to do it yourself by loading them using `numpy`, but there is a util `utils/output.py` that conveniently handles that for you, as we will see below. Once you get familiar with the outputs, nothing stands between you and unlocking the full potential of the `JADE` code.

## Use case FD1: Forward modeling, dynamics alone

The most immediate use you can make of the `JADE` code is to simulate the dynamical evolution of a hierarchical three-body system. This corresponds to the **FD1** use case.

The `JADE` code has been developed first and foremost for modeling such a configuration, in order to be able to monitor the evolution of a von Zeipel–Lidov–Kozai (ZLK) resonance (see [von Zeipel 1910](https://ui.adsabs.harvard.edu/abs/1910AN....183..345V/abstract), [Lidov 1962](https://ui.adsabs.harvard.edu/abs/1962P%26SS....9..719L/abstract), [Kozai 1962](https://ui.adsabs.harvard.edu/abs/1962AJ.....67..591K/abstract) for the historical publications, [Naoz 2016](https://ui.adsabs.harvard.edu/abs/2016ARA%26A..54..441N/abstract) for a comprehensive review, and [Attia et al. 2021](https://ui.adsabs.harvard.edu/abs/2021A%26A...647A..40A/abstract) for its application to `JADE`). By saying hierarchical three-body system, we mean here two bodies (typically an inner planet and an outer perturber) orbiting a central one (typically a star) at semi-major axes $a$ and $a_{\rm pert}$, such as $a_{\rm pert} \gg a$. 

Let us run a simulation featuring a **ZLK resonance counterbalanced by tidal effects**, which corresponds to a typical dynamical evolution you would want to model. An example input file is provided to this end: `input/examples/example_fd1.txt`. First, have a look at this file to assess its contents. In particular, you will see the following settings at the beginning of the file:
- `dyn = True`: the dynamical integrator is turned on.
- `perturber = True`: the action of the third body (perturber) is turned on.
- `orderdyn = 4`: the secular equations of the perturber are truncated to the fourth (hexadecapolar) order. This is the highest order, you could also set `2` or `3`. The higher you go, the more accurate your dynamical evolution will be, but the slower the simulation will be. Physical differences related to using different orders can be found in, e.g., [Naoz (2016)](https://ui.adsabs.harvard.edu/abs/2016ARA%26A..54..441N/abstract).
- `tides = True`: star–planet tidal dissipation is turned on.
- `atmo = False`: the atmospheric structure/evaporation integrators are turned off.

The temporal bounds of the simulation are set by these two parameters:
- `t_init = 10.`: the simulation will start at $t_{\rm i} = 10\,{\rm Myr}$, typical disk dispersal time.
- `age = 100.`: the simulation will stop at $t_{\rm f} = 100\,{\rm Myr}$.

Further in the input file, you can see that we will model a Sun-like star, orbited by an initially circular warm mini-Neptune perturbed by an inclined, circular cold Jupiter-size planet:
- `Ms = 1.` and `Rs = 1.`: the stellar mass and radius are $M_\star = 1\,M_\odot$ and $R_\star = 1\,R_\odot$.
- `Mpl = 0.06` and `Rpl = 0.337`: the inner planet mass and radius are $M_{\rm p} = 0.06\,M_{\rm J} \simeq 19\,M_\oplus$ and $R_{\rm p} = 0.337\,R_{\rm J} \simeq 3.8\,R_\oplus$.
- `Mpert = 1.`: the perturber mass is $M_{\rm pert} = 1\,M_{\rm J}$. There is no perturber radius (point mass).
- `planet_sma = 0.5`: the initial semi-major axis of the inner planet is $a(t = t_{\rm i}) = 0.5\,{\rm AU}$.
- `pert_sma = 10.`: the semi-major axis of the perturber is $a_{\rm pert} = 10\,{\rm AU}$ (fixed). You can see that the hierarchical condition is satisfied (you will be fine as long as `pert_sma` $> 10\,\times$ `planet_sma`).
- `planet_ecc = 0.00001`: the initial eccentricity of the inner planet is $e(t = t_{\rm i}) = 10^{-5} \simeq 0$ (not exactly set to zero for numerical reasons).
- `pert_ecc = 0.00001`: the eccentricity of the perturber is $e_{\rm pert} = 10^{-5} \simeq 0$ (fixed, and also not exactly set to zero for numerical reasons).

**Important note on the mutual inclination:** you probably want to set an initial mutual inclination $i_{\rm mut}(t = t_{\rm i})$ that is high enough to trigger a ZLK resonance ($> 39.2^\circ$ for circular orbits, e.g., [Kinoshita & Nakai 1999](https://ui.adsabs.harvard.edu/abs/1999CeMDA..75..125K/abstract)), and in general you probably want to have control over the initial mutual inclination. However, transforming it into the right initial orbital parameters defined at the end of the input file can be a real headache. Fortunately, by aligning the lines of nodes of the planet and perturber, as is done in the example, it becomes trivial. This is why
- `planet_lambd = 0.` and `pert_lambd = 0.`: the mean longitudes are $\lambda(t = t_{\rm i}) = \lambda_{\rm pert} = 0^\circ$,
- `planet_omega = 90.` and `pert_omega = 90.`: the longitudes of periastra are $\varpi(t = t_{\rm i}) = \varpi_{\rm pert} = 90^\circ$,
- `planet_Omega = 90.` and `pert_Omega = 90.`: the longitudes of ascending nodes are $\Omega(t = t_{\rm i}) = \Omega_{\rm pert} = 90^\circ$.

This configuration ensures that $i_{\rm mut}(t = t_{\rm i}) = \lvert i(t = t_{\rm i}) - i_{\rm pert} \rvert$, which allows you to set the initial mutual inclination directly from the orbital inclinations. In the example, as we have `planet_incl = 90.` and `pert_incl = 15.`, we hence get $i_{\rm mut}(t = t_{\rm i}) = 75^\circ$.

*Note:* this configuration also ensures that the initial argument of periastron of the inner planet is $\omega(t = t_{\rm i}) = 0^\circ$, which is the locus of minimum eccentricity in a ZLK resonance (e.g., [Naoz 2016](https://ui.adsabs.harvard.edu/abs/2016ARA%26A..54..441N/abstract)), consistent with the initial circular orbit.

***

You can now launch the simulation. Once you make sure you are in the `routines/` folder, run the following command in a terminal:

    python main.py examples/example_fd1.txt
    
The `JADE` code being optimized for simulating ZLK resonances (in particular through the use of a `C` backend for the dynamical integrator), this should not take more than a few tens of seconds. The elapsed time will be shown on the log, as well as various other information.

Let us analyze the simulation output. We will employ the `utils/output.py` util to this end. You can either create a new folder at the base of the package (e.g., `analyses/`) and copy/paste `output.py` there to import it easily in a notebook you create there (preferred solution for keeping a clear structure), or use the `sys.path.insert` trick like below (or even directly create a notebook in `utils/` if you prefer).

In [ ]:
# Importing necessary packages
import sys
import matplotlib.pyplot as plt

# Importing the output util
sys.path.insert(1, '../utils/')
from output import JADE_output

# Unit conversion constants
MJ2E = 317.8284065946748 #[M_Jup] to [M_Earth]
RJ2E = 11.20898073093868 #[R_jup] to [R_Earth]

The `JADE_output` class of `output.py` is what you will use for handling the `.npz` files. You can find its documentation directly inside the util, including its class attributes and functions. **It is important to read the documentation and get familiar with it before proceeding**. The basic usage of the util is the following:

1. Create a `JADE_output` instance. The easiest way to call the constructor is to set the two arguments defining the input file and the output folder (containing the `.npz` files), respectively the `txt` and `npz` kwargs.
2. Set a uniform time variable for all the parameters using the `set_time()` function. This will be particularly relevant when we will make use of both the dynamical and atmospheric integrators, as they are not necessarily called at the same time variables during a simulation.
3. Generate additional parameters that are not automatically created by the constructor through the `generate(*args)` function. Again, please refer to the documentation to know the parameters that are created by default and the ones you can generate using this function.

In [ ]:
# Defining the input file and output folder paths
input_file_fd    = '../input/examples/example_fd1.txt'
output_folder_fd = '../saved_data/examples/example_fd1/'

# Creating a JADE_output instance
sim_fd = JADE_output(txt=input_file_fd, npz=output_folder_fd, verbose=True)

# Setting a uniform time variable
sim_fd.set_time()

# Generating additional parameters, here the mutual inclination 'imut' and the spin-orbit angle 'psi'
sim_fd.generate('imut', 'psi')

We will now plot the temporal evolution of the inner planet's semi-major axis $a$, eccentricity $e$, mutual inclination with the perturber $i_{\rm mut}$, and spin–orbit angle $\psi$.

All the parameters, including the ones you created using `generate()`, can be accessed as class attributes now. See the documentation for their units.

In [ ]:
# Storing the parameters to be plotted

t_fd   = sim_fd.t*1e-6 #Time [Myr]
a_fd   = sim_fd.a      #Semi-major axis [AU]
e_fd   = sim_fd.e      #Eccentricity
i_fd   = sim_fd.imut   #Mutual inclination [deg]
psi_fd = sim_fd.psi    #Spin-orbit angle [deg]

In [ ]:
# Plotting

fig, ax = plt.subplots(2, 2, figsize=(10, 4), sharex=True, dpi=300)

ax[0][0].plot(t_fd, a_fd)
ax[0][1].plot(t_fd, e_fd)
ax[1][0].plot(t_fd, psi_fd)
ax[1][1].plot(t_fd, i_fd)

ax[1][0].set_xlabel('Time [Myr]')
ax[1][1].set_xlabel('Time [Myr]')
ax[0][0].set_ylabel(r'$a$ [AU]')
ax[0][1].set_ylabel(r'$e$')
ax[1][0].set_ylabel(r'$\psi$ [$^\circ$]')
ax[1][1].set_ylabel(r'$i_{\rm mut}$ [$^\circ$]')

fig.tight_layout()
plt.show()

As you can see, this is the **typical ZLK resonance counterbalanced by tidal effects**, like announced:

- Large eccentricity and mutual inclination oscillations in opposite directions (ZLK), with a shrinking amplitude (tides).
- Staircase-shaped semi-major axis due to the large impact of tides at eccentricity peaks.
- Widely oscillating spin–orbit angle, in a complex manner.

## Use cases FA3, FC: Forward modeling, dynamics and atmosphere coupled

The benefit of use the `JADE` code, compared to other secular dynamical integrators, is that it is precisely not a mere secular dynamical integrator. We will activate the atmospheric structure/evaporation integrators to **assess the erosion status** of our example mini-Neptune: this is the **FA3** use case. 

In parallel, the dynamical integrator will remain active, in order to **monitor the long-term interplay between a ZLK resonance and atmospheric evaporation**, which corresponds to the **FC** use case. Using the `JADE` code in this way represents the core of why you may want to employ it.

An example input file is provided in `input/examples/example_fc.txt`, which builds upon our previous example and adds the atmospheric part to the simulation. Again, have a thorough look at it before proceeding. In particular, you will see that the following settings at the beginning of the input file have changed:

- `atmo = True`: the atmospheric structure integrator is turned on.
- `evap = True`: the evaporation integrator is turned on.
- `parallel = 5`: five cores are used for the atmospheric structure integrator. 

*Note:* in the `parallel` field, we are indeed talking about *cores* here, not *threads*. Of course, the higher this number is, the faster the structure integrator will be. However, make sure you do not overload all your computer cores by setting a too-high number. Furthermore, extensive experience with `JADE` recommends setting an **odd number** of cores, for numerical reasons.

You can notice that activating evaporation is straightforward, you just have to switch on the `evap` setting. You may raise a brow reading this, since you would usually need to specify other parameters such as the evaporation efficiency ($\epsilon$, as is customary within the energy-limited formalism, e.g., [Lopez 2017](https://ui.adsabs.harvard.edu/abs/2017MNRAS.472..245L/abstract)). Nonetheless, the `JADE` code self-consistently computes all the necessary quantities from the system properties, in a dynamical manner (using the formalism of [Salz et al. 2016](https://ui.adsabs.harvard.edu/abs/2016A%26A...585L...2S/abstract)), which ensures a better agreement with observed mass-loss rates than a naive energy-limited approach.

Still, the stellar radiative input has to be specified:

- `stellar_lum = analytic`: the stellar luminosity is computed using analytic formulae (see [Attia et al. 2021](https://ui.adsabs.harvard.edu/abs/2021A%26A...647A..40A/abstract)).
- `Lbol = 4e33`: the stellar bolometric luminosity is $L_\star = 4 \times 10^{33}\,{\rm erg/s} \simeq 1\,L_\odot$. 

You have to define a value at least for `Lbol` in the `analytic` mode, the other parameters `LX_Lbol_sat`, `tau_X_bol_sat`, and `alpha_X_bol` having default values if you leave them blank. You could also set the `stellar_lum = tabular` mode, in which case you have to provide a table with stellar bolometric and XUV luminosities as a function of time to be interpolated during the simulation. In that case, it is not necessary to define the stellar luminosity parameters (except for the `stellar_lum_path` of course, which has to be relative to `luminosities/`). **Luminosity files should always be located in the `luminosities/` folder**. An example luminosity file `luminosities/example.txt` is provided to show you the correct format.

The system parameters remain globally unchanged compared to the FD1 example, with the exception of the following:

- `Mcore = 0.054`: the mass of the iron nucleus and silicate mantle of the inner planet is $M_{\rm core} = 0.054\,M_{\rm J} = 0.9\,M_{\rm p}$, meaning that the atmosphere represents 10% of the total mass.
- `Rpl = `: the inner planet radius is automatically computed by the structure integrator.
- `YHe = 0.2`: the atmospheric mass fraction of helium is $Y = 0.2$.
- `Zmet = 0.0001`: the atmospheric mass fraction of metals is $Z = 10^{-4}$.
- `fmantle = 0.6`: the silicate mass fraction within $M_{\rm core}$ is $f_{\rm Si/core} = 0.6$.

**Important note on the metallicity:** `Zmet` should always be set (for now) to a small value ($Z \leq 10^{-4}$) so as to be a trace amount that does not affect the atmospheric mean molecular weight. It will still restore the atmospheric opacity to physical values (see discussion in [Attia et al. 2025]()).

***

You can now launch the simulation. Once you make sure you are in the `routines/` folder, run the following command in a terminal:

    python main.py examples/example_fc.txt
    
The structure integrator of `JADE` is clearly its computational bottleneck. The simulation should instead run in a couple of minutes, compared to the couple of tens of seconds of the FD1 example.

Once the simulation is finished, let us analyze its results in the same way as before.

In [ ]:
# Defining the input file and output folder paths
input_file_fc    = '../input/examples/example_fc.txt'
output_folder_fc = '../saved_data/examples/example_fc/'

# Creating a JADE_output instance
sim_fc = JADE_output(txt=input_file_fc, npz=output_folder_fc, verbose=True)

# Setting a uniform time variable
sim_fc.set_time()

# Generating additional parameters, here the spin-orbit angle 'psi',
# the equilibirum temperature 'Teq', and the intrinsic temperature 'Tint'
sim_fc.generate('psi', 'Teq', 'Tint')

In [ ]:
# Storing the parameters to be plotted

t_fc    = sim_fc.t*1e-6  #Time [Myr]
a_fc    = sim_fc.a       #Semi-major axis [AU]
e_fc    = sim_fc.e       #Eccentricity
psi_fc  = sim_fc.psi     #Spin-orbit angle [deg]
M_fc    = sim_fc.Mp*MJ2E #Mass [M_Earth]
R_fc    = sim_fc.Rp*RJ2E #Radius [R_Earth]
Teq_fc  = sim_fc.Teq     #Equilibrium temperature [K]
Tint_fc = sim_fc.Tint    #Intrinsic temperature [K]

We will also overplot the results of the FD1 example, so as to visualize the impact of accounting for evaporation and an accurately derived planet radius. Hence, we need the mass and radius from the previous simulation as well (even if they were constant).

In [ ]:
M_fd = sim_fd.Mp*MJ2E #Mass [M_Earth]
R_fd = sim_fd.Rp*RJ2E #Radius [R_Earth]

In [ ]:
# Plotting

fig, ax = plt.subplots(3, 2, figsize=(10, 6), sharex=True, dpi=300)

ax[0][0].plot(t_fd, a_fd,    c='lightblue', label='FD1')
ax[0][1].plot(t_fd, e_fd,    c='lightblue')
ax[1][0].plot(t_fd, M_fd,    c='lightblue')
ax[1][1].plot(t_fd, R_fd,    c='lightblue')
ax[2][0].plot(t_fd, psi_fd,  c='lightblue')

ax[0][0].plot(t_fc, a_fc,    c='C0',        label='FC')
ax[0][1].plot(t_fc, e_fc,    c='C0')
ax[1][0].plot(t_fc, M_fc,    c='C0')
ax[1][1].plot(t_fc, R_fc,    c='C0')
ax[2][0].plot(t_fc, psi_fc,  c='C0')
ax[2][1].plot(t_fc, Teq_fc,  c='hotpink',   label=r'$T_{\rm eq}$')
ax[2][1].plot(t_fc, Tint_fc, c='limegreen', label=r'$T_{\rm int}$')

ax[0][0].legend()
ax[2][1].legend()

ax[2][0].set_xlabel('Time [Myr]')
ax[2][1].set_xlabel('Time [Myr]')
ax[0][0].set_ylabel(r'$a$ [AU]')
ax[0][1].set_ylabel(r'$e$')
ax[1][0].set_ylabel(r'$M_{\rm p}$ [$M_\oplus$]')
ax[1][1].set_ylabel(r'$R_{\rm p}$ [$R_\oplus$]')
ax[2][0].set_ylabel(r'$\psi$ [$^\circ$]')
ax[2][1].set_ylabel(r'$T$ [K]')

fig.tight_layout()
plt.show()

Some notable features can be underlined here, especially the **fingerprints of the coupling between dynamical and atmospheric evolution**:

- The planet mass dropped because the atmosphere suffered evaporation.
- The planet radius showcases **radius pulsations in tune with the ZLK cycles**, a phenomenon first identified by `JADE` (and well described in [Attia et al. 2021](https://ui.adsabs.harvard.edu/abs/2021A%26A...647A..40A/abstract)) due to the **dynamical feedback on the atmosphere**: eccentricity peaks are responsible for a higher irradiation, translating into a periodic heating of the outer layers of the atmosphere (as also seen in the $T_{\rm eq}$ pulsations).
- **The eccentricity amplitude shrinks faster** than for the FD1 case due to the **atmospheric feedback on the dynamics**: as the radius pulsates, tides are even stronger than before at eccentricity peaks. This will eventually result in a faster decoupling of the inner planet from the perturber, when the eccentricity amplitude cancels out (a phenomenon also first identified by `JADE` and well described in [Attia et al. 2021](https://ui.adsabs.harvard.edu/abs/2021A%26A...647A..40A/abstract)).

## Use case FA1: Forward modeling, atmospheric structure

To finish this tutorial, let us now have a look at the **atmospheric and interior structure** of our example warm mini-Neptune, which represents the **FA1** use case. We invite you to read Sect. 2.3.1 of [Attia et al. (2021)](https://ui.adsabs.harvard.edu/abs/2021A%26A...647A..40A/abstract), and particularly check its Fig. 5, to make sure you understand how the planet interior is layered in `JADE`:

- *Zone A*, top layer where the incident stellar flux is collected, at low optical depths.
- *Zone B*, intermediate layer where heat is redistributed by convection and radiation.
- *Core*, solid material comprised of an iron nucleus surrounded by a silicate mantle, described by isothermal equations of state.

There is no need to launch any simulation for FA1. The `output.py` util can handle it on its own based on an existing simulation, from which it will **construct atmospheric profiles at any time variable**. We will continue working with our previous FC example for this part.

***

We will employ the `atmo_profiles()` function of the `JADE_output` class for this, which takes as an argument the time, in ${\rm yr}$, at which the atmospheric profiles should be constructed. It returns the following ones, in this order:

- Optical depth profile $\tau$, defined only in Zone A.
- Mass profile $m_{\rm p}$, defined everywhere.
- Radius profile $r_{\rm p}$, defined everywhere.
- Temperature profile $T$, defined only in Zone A and Zone B.
- Pressure profile $P$, defined everywhere.

Let us generate these profiles at $t_{\rm i}$ and $t_{\rm f}$, the temporal bounds of our simulation.

In [ ]:
# Generating atmospheric profiles at t = t_i
tau_fc_i, m_fc_i, r_fc_i, T_fc_i, P_fc_i = sim_fc.atmo_profiles(1e7)

# Generating atmospheric profiles at t = t_f (default time if no argument is provided)
tau_fc_f, m_fc_f, r_fc_f, T_fc_f, P_fc_f = sim_fc.atmo_profiles()

We will segment the atmospheric profiles into Zone A, Zone B, and Core, so as to correctly visualize their differences and the transition between them. 

Since $\tau$ is only defined in Zone A, we will use `len(tau_fc_i)` and `len(tau_fc_f)` to delimit this region.

We will do the same with $T$, defined in Zone A and Zone B, allowing us to employ `len(T_fc_i)` and `len(T_fc_f)` to identify the end of the latter region.

In [ ]:
# Unit conversion

m_fc_i *= MJ2E #Total mass profile at t = t_i [M_Earth]
r_fc_i *= RJ2E #Total radius profile at t = t_i [R_Earth]
m_fc_f *= MJ2E #Total mass profile at t = t_f [M_Earth]
r_fc_f *= RJ2E #Total radius profile at t = t_f [R_Earth]

#--------------------------------
# Atmospheric profiles at t = t_i

# Delimiting Zone A and Zone B
n_za_i = len(tau_fc_i)
n_zb_i = len(T_fc_i)

# Zone A
P_za_i = P_fc_i[:n_za_i]       #Pressure [bar]
m_za_i = m_fc_i[:n_za_i]       #Mass [M_Earth]
r_za_i = r_fc_i[:n_za_i]       #Radius [R_Earth]
T_za_i = T_fc_i[:n_za_i]       #Temperature [K]

# Zone B
P_zb_i = P_fc_i[n_za_i:n_zb_i] #Pressure [bar]
m_zb_i = m_fc_i[n_za_i:n_zb_i] #Mass [M_Earth]
r_zb_i = r_fc_i[n_za_i:n_zb_i] #Radius [R_Earth]
T_zb_i = T_fc_i[n_za_i:]       #Temperature [K]

# Solid core
P_sc_i = P_fc_i[n_zb_i:]       #Pressure [bar]
m_sc_i = m_fc_i[n_zb_i:]       #Mass [M_Earth]
r_sc_i = r_fc_i[n_zb_i:]       #Radius [R_Earth]

#--------------------------------
# Atmospheric profiles at t = t_f

# Delimiting Zone A and Zone B
n_za_f = len(tau_fc_f)
n_zb_f = len(T_fc_f)

# Zone A
P_za_f = P_fc_f[:n_za_f]       #Pressure [bar]
m_za_f = m_fc_f[:n_za_f]       #Mass [M_Earth]
r_za_f = r_fc_f[:n_za_f]       #Radius [R_Earth]
T_za_f = T_fc_f[:n_za_f]       #Temperature [K]

# Zone B
P_zb_f = P_fc_f[n_za_f:n_zb_f] #Pressure [bar]
m_zb_f = m_fc_f[n_za_f:n_zb_f] #Mass [M_Earth]
r_zb_f = r_fc_f[n_za_f:n_zb_f] #Radius [R_Earth]
T_zb_f = T_fc_f[n_za_f:]       #Temperature [K]

# Solid core
P_sc_f = P_fc_f[n_zb_f:]       #Pressure [bar]
m_sc_f = m_fc_f[n_zb_f:]       #Mass [M_Earth]
r_sc_f = r_fc_f[n_zb_f:]       #Radius [R_Earth]

In [ ]:
# Plotting 

fig, ax = plt.subplots(2, 2, figsize=(10, 8), dpi=300)

ax[0][0].plot(m_za_i,   P_za_i, c='dodgerblue', ls='--')
ax[0][0].plot(m_zb_i,   P_zb_i, c='orange',     ls='--')
ax[0][0].plot(m_sc_i,   P_sc_i, c='black',      ls='--')
ax[0][0].plot(m_za_f,   P_za_f, c='dodgerblue',          label='Zone A')
ax[0][0].plot(m_zb_f,   P_zb_f, c='orange',              label='Zone B')
ax[0][0].plot(m_sc_f,   P_sc_f, c='black',               label='Core')

ax[0][1].plot(r_za_i,   P_za_i, c='dodgerblue', ls='--')
ax[0][1].plot(r_zb_i,   P_zb_i, c='orange',     ls='--')
ax[0][1].plot(r_sc_i,   P_sc_i, c='black',      ls='--')
ax[0][1].plot(r_za_f,   P_za_f, c='dodgerblue')
ax[0][1].plot(r_zb_f,   P_zb_f, c='orange')
ax[0][1].plot(r_sc_f,   P_sc_f, c='black')

ax[1][0].plot(T_za_i,   P_za_i, c='dodgerblue', ls='--', label='Start')
ax[1][0].plot(T_zb_i,   P_zb_i, c='orange',     ls='--')
ax[1][0].plot(T_za_f,   P_za_f, c='dodgerblue',          label='End')
ax[1][0].plot(T_zb_f,   P_zb_f, c='orange')

ax[1][1].plot(tau_fc_i, P_za_i, c='dodgerblue', ls='--')
ax[1][1].plot(tau_fc_f, P_za_f, c='dodgerblue')

for _ax in ax.flatten(): 
    _ax.semilogy()
    _ax.invert_yaxis()
ax[1][1].semilogx()

ax[0][0].legend()
ax[1][0].legend()

for _ax in ax.flatten(): 
    _ax.set_ylabel(r'$P$ [bar]')
ax[0][0].set_xlabel(r'$m_{\rm p}$ [$M_\oplus$]')
ax[0][1].set_xlabel(r'$r_{\rm p}$ [$R_\oplus$]')
ax[1][0].set_xlabel(r'$T$ [K]')
ax[1][1].set_xlabel(r'$\tau$')

fig.tight_layout()
plt.show()

***

**Congratulations, you have reached the end of this first tutorial**. You know now how to use the fundamental blocks of the `JADE` code. All the other, more advanced, use cases covered by the following tutorials build upon what you saw here. We hope this tutorial was of help, and wish you great fun with doing science using the `JADE` code.